In [ ]:
import pandas as pd
import altair as alt

In [ ]:
full_df = pd.read_csv("../data/goodreads_library_export.csv")

In [ ]:
full_df.columns

In [ ]:
relevant_cols = ["Book Id", 
        "Title", 
        "Author", 
        "ISBN", 
        "ISBN13", 
        "My Rating", 
        "Average Rating", 
        "Publisher", 
        "Number of Pages", 
        "Year Published",
        "Date Read",
        "Date Added",
        "Bookshelves",
        "Exclusive Shelf",
        "Read Count"
       ]

cols = {}
for col in relevant_cols:
    cols[col] = col.replace(" ", "_").lower()

In [ ]:
df = full_df[cols_dict.keys()].rename(columns=cols)

In [ ]:
df[["isbn", "isbn13"]] = df[["isbn", "isbn13"]].replace(regex='["=]', value="")

In [ ]:
df = df.astype({"date_read": "datetime64", "date_added": "datetime64"})

In [ ]:
alt.Chart(df).mark_bar().encode(
    x="count()",
    y="exclusive_shelf"
)

In [ ]:
alt.Chart(df.loc[df["exclusive_shelf"] == "read"]).transform_filter(
    "year(datum.date_read) >= 2014"
).mark_bar().encode(
    x="yearmonth(date_read):T",
    y="count()"
).properties(width=800)

In [ ]:
df[df.duplicated(subset="title", keep=False)].sort_values(by="title")

In [ ]:
test = [0,1,3,2,5,3,2,6,1,0,12,1,2,7,8]


In [ ]:
%timeit result = [i for i in test if test.count(i)==1]

In [ ]:
%%timeit
result2 = []

for i in test:
    if test.count(i) == 1:
        result2.append(i)

In [ ]:
%%timeit
features_count = {i: test.count(i) for i in test}
single_features = [key for key, value in features_count.items() if value == 1]

In [ ]:
result == result2 == single_features